CODE BLOCK TO EXTRACT WEATHER FROM LAT LONG

In [20]:
import pandas as pd
import reverse_geocode as rg
import numpy as np

In [21]:
from datetime import datetime,timedelta
from astropy.time import Time
from meteostat import Point, Hourly

In [22]:
LIMITED = False
METEOSTAT = False
LINUX = False

In [23]:
data_file = '..\\Data\\moon_sighting_data.csv'
if LINUX:
    data_file = '../Data/moon_sighting_data.csv'

data = pd.read_csv(data_file)

data = data[data["Source"]=="ICOUK"]


#data = data[['Latitude', "Longitude", 'Date']]
if LIMITED:
    df = data.copy('Deep').head(20)
else:
    df = data.copy('Deep')

In [24]:
#index used
i =334

In [25]:
#get city from lat long, not rlly needed
# city_arr = []
# city_dic = rg.search(df[['Latitude', 'Longitude']])
# for i in range(len(city_dic)):
#     city_arr.append((city_dic[i])['city']+','+(city_dic[i])['country_code'])

# df['City'] = city_arr

In [26]:
#df

In [27]:
if METEOSTAT:
    # adds meteostat weather data for dataset

    weather_data =[]
    for i in range(len(city_dic)):
        day = Time(df['Date'][i],format='jd').to_datetime()
        weather_data_temp = Hourly(Point(df['Latitude'][i],df['Longitude'][i]), day-timedelta(hours=0.5), day + timedelta(hours=0.5)) #day+timedelta(days=1))
        x = weather_data_temp.fetch()
        if (len(x)>0):
            weather_data.append(x)
        else:
            weather_data.append( pd.DataFrame(np.zeros([1, 11])*np.nan, columns = ['temp', 'dwpt', 'rhum', 'prcp', 'snow','wdir ','wspd','wpgt','pres', 'tsun', 'coco']))
    weather_dataset = (pd.concat(weather_data, ignore_index=True)).drop(['wdir','wspd', 'wpgt'], axis=1)

In [28]:
import requests
from bs4 import BeautifulSoup
import re


In [29]:
best_time = Time(df['Date'][i], format='jd') #best time in julian
DATE = str(int(np.round(best_time.unix/60/30)*60*30)) #convert julian date to UNIX timestamp HOWEVER has british summertime
print(DATE)
url = 'https://www.weatheronline.co.uk/weather/maps/current?LANG=en&DATE='+ DATE  +'&CONT=ukuk&LAND=UK&KEY=UK&SORT=4&UD=0&INT=06&TYP=bedeckung&ART=tabelle&RUBRIK=akt&R=310&CEL=C&SI=mph'
response = requests.get(url)
print(response.status_code)

1613151000
200


In [30]:
soup = BeautifulSoup(response.text, 'html.parser')

In [31]:
table = soup.find('table', class_ = 'gr3')

table = str(table)
#print(table)

In [32]:
names = re.findall("(?<=CEL=C\">)(.*?)(?= \()", table)
code = re.findall("(?<=WMO=)(.*?)(?=\&amp)", table)
cloud_level = re.findall("(?<=<td>)(\d)(?=<\/td>)", table)
print(names)
print(code)
print(cloud_level)

['Camborne', 'Lerwick', 'Northolt', 'RAF Brize Norton', 'Lossiemouth', 'Coningsby', 'Isle of Man Airport', 'MoD Boscombe Down', 'Guernsey Airport', 'Yeovil', 'RAF Odiham']
['03808', '03005', '03672', '03649', '03068', '03391', '03204', '03746', '03894', '03853', '03761']
['7', '7', '6', '6', '4', '3', '3', '1', '1', '1', '1']


METEOSTAT distance comparison

In [33]:
from meteostat import Stations

In [34]:
day = best_time.to_datetime()
print(day)

2021-02-12 17:31:40.512014


In [35]:
stations = Stations().nearby(df['Latitude'][i],df['Longitude'][i])
stations = stations.inventory('hourly', day)
station = stations.fetch(10)

In [36]:
station

,name,country,region,wmo,icao,latitude,longitude,elevation,timezone,hourly_start,hourly_end,daily_start,daily_end,monthly_start,monthly_end,distance
id,,,,,,,,,,,,,,,,
03334,Manchester Airport,GB,ENG,03334,EGCC,53.3500,-2.2833,69.0,Europe/London,1933-09-30,2023-11-17,1973-01-01,2022-04-25,1957-01-01,2022-01-01,27780.788685
03318,Blackpool Airport,GB,ENG,03318,EGNH,53.7667,-3.0333,10.0,Europe/London,1973-01-01,2023-11-17,1973-01-02,2022-04-24,1981-01-01,2022-01-01,44468.650060
03321,Hawarden,GB,WLS,03321,EGNR,53.1667,-2.9833,10.0,Europe/London,1985-09-04,2023-11-17,1985-09-05,2022-04-24,2005-01-01,2022-01-01,58967.443229
EGNM0,Leeds And Bradford / Yeadon,GB,ENG,<NA>,EGNM,53.8667,-1.6500,208.0,Europe/London,1988-01-05,2023-11-17,1995-05-11,2022-04-27,2006-01-01,2022-01-01,60403.951143
03214,Walney Island,GB,CU,03214,EGNL,54.1167,-3.2500,15.0,Europe/London,1983-08-04,2023-10-06,1983-08-11,2022-03-25,2005-01-01,2021-01-01,79917.927119
03414,Shawbury,GB,ENG,03414,EGOS,52.8000,-2.6667,76.0,Europe/London,1973-01-01,2023-11-17,1973-01-01,2023-11-11,1973-01-01,2022-01-01,88478.882452
03266,Linton-On-Ouse,GB,ENG,03266,EGXU,54.0500,-1.2500,16.0,Europe/London,1973-01-01,2022-07-04,1973-01-09,2021-08-31,2000-01-01,2021-01-01,93416.895331
03264,Topcliffe / Dalton,GB,ENG,03264,EGXZ,54.2000,-1.3833,28.0,Europe/London,1994-07-20,2023-11-17,2005-08-24,2022-04-24,2010-01-01,2022-01-01,97138.834806
03257,Leeming,GB,ENG,03257,EGXE,54.3000,-1.5333,40.0,Europe/London,1973-01-01,2023-11-17,1973-01-01,2023-11-10,1971-01-01,2022-01-01,99097.026218


In [37]:
#finds the first agreement of metrostat and weather online to give closest station
closest_name = station['name'][np.where(np.in1d(station.index, code))[0][0]]
print(closest_name)
cloud_level[np.where(np.in1d(names, closest_name))[0][0]]

IndexError: index 0 is out of bounds for axis 0 with size 0